In [66]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import itertools
from util.save_load import load_kernel_model
from dataset.graphs_kernel import get_graph_data
from kernels.wrapper import MODELS, KernelModelWrapper
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, get_number_of_ipc2023_training_data
from itertools import product
from IPython.display import display, HTML

In [67]:
_LOG_DIR = "logs/train_kernel"

ITERATIONS = list(range(1, 10))

PRUNE_COEFS = list(range(10))

N_TRAINING_DATA = get_number_of_ipc2023_training_data()

### Train metrics

In [68]:
def get_data(domain):
  d = {
    "config": [],
    "mse": [],
    "f1": [],
    "nonzero_weights": [],
  }

  CONFIGS = list(product(MODELS, ITERATIONS, PRUNE_COEFS))
  for config in CONFIGS:
    model, iterations, prune = config
    log_file = "_".join([model, "llg", "ipc2023-learning-"+domain, "wl", str(iterations), str(prune*iterations)])+".log"
    log_file = _LOG_DIR + "/" + log_file
    
    if not os.path.exists(log_file):
      continue

    stats = {
      "config": "_".join([model, "wl", str(iterations), str(prune*iterations)])
    }

    lines = list(open(log_file, 'r').readlines())
    for line in lines:
      toks = line.split()
      if "train_mse" in line:
        stats["mse"] = float(toks[-1])
      elif "train_f1_macro" in line:
        stats["f1"] = float(toks[-1])
      elif "zero_weights" in line:
        weights = int(toks[1].split('/')[1])
        zeros = int(toks[1].split('/')[0])
        stats["nonzero_weights"] = weights - zeros
    
    if "nonzero_weights" not in stats:
      stats["nonzero_weights"] = "na"

    if len(stats) != len(d):
      continue

    for key in stats:
      d[key].append(stats[key])

  return d

def get_df(domain):
  d = get_data(domain)
  return pd.DataFrame(d)

In [69]:
get_data("blocksworld")


{'config': ['linear-svr_wl_1_0',
  'linear-svr_wl_1_1',
  'linear-svr_wl_1_2',
  'linear-svr_wl_1_3',
  'linear-svr_wl_1_4',
  'linear-svr_wl_1_5',
  'linear-svr_wl_1_6',
  'linear-svr_wl_1_7',
  'linear-svr_wl_1_8',
  'linear-svr_wl_1_9',
  'linear-svr_wl_2_0',
  'linear-svr_wl_2_2',
  'linear-svr_wl_2_4',
  'linear-svr_wl_2_6',
  'linear-svr_wl_2_8',
  'linear-svr_wl_2_10',
  'linear-svr_wl_2_12',
  'linear-svr_wl_2_14',
  'linear-svr_wl_2_16',
  'linear-svr_wl_2_18',
  'linear-svr_wl_3_0',
  'linear-svr_wl_3_3',
  'linear-svr_wl_3_6',
  'linear-svr_wl_3_9',
  'linear-svr_wl_3_12',
  'linear-svr_wl_3_15',
  'linear-svr_wl_3_18',
  'linear-svr_wl_3_21',
  'linear-svr_wl_3_24',
  'linear-svr_wl_3_27',
  'linear-svr_wl_4_0',
  'linear-svr_wl_4_4',
  'linear-svr_wl_4_8',
  'linear-svr_wl_4_12',
  'linear-svr_wl_4_16',
  'linear-svr_wl_4_20',
  'linear-svr_wl_4_24',
  'linear-svr_wl_4_28',
  'linear-svr_wl_4_32',
  'linear-svr_wl_4_36',
  'linear-svr_wl_5_0',
  'linear-svr_wl_5_5',
  'lin

In [70]:
for domain in IPC2023_LEARNING_DOMAINS:
  for model in ["linear-svr", "lasso", "ridge"]:
    d = get_data(domain)

    map_scores = {}
    for k, config in enumerate(d['config']):
      if model not in config:
        continue
      toks = config.split("_")
      key = (int(toks[-2]), int(toks[-1]))
      map_scores[key] = {
        "mse": d['mse'][k],
        "f1": d['f1'][k],
        "nonzero_weights": d['nonzero_weights'][k],
      }

    fig, ax = plt.subplots(1, 3)
    for k, metric in enumerate(["f1", "mse", "nonzero_weights"]):
      heatmap = np.zeros((len(ITERATIONS), len(PRUNE_COEFS)))
      for i in ITERATIONS:
        for j in PRUNE_COEFS:
          try:
            heatmap[i-1,j] = map_scores[(i, j*i)][metric]
            if metric=='nonzero_weights':
              heatmap[i-1,j] = np.log10(heatmap[i-1,j])
          except:
            heatmap[i-1,j] = np.nan

      ax[k].set_xticks(range(len(PRUNE_COEFS)), PRUNE_COEFS)
      ax[k].set_xlabel("Prune Coefficient")
      ax[k].set_yticks(range(len(ITERATIONS)), ITERATIONS)
      ax[k].set_ylabel("WL iterations")
      ax[k].imshow(heatmap)
      # ax[k].colorbar()
      if metric == "nonzero_weights":
        title = (f"nonzero_weights log10")
      else:
        title = metric
      ax[k].set_title(title)
    plt.title(f"{model} {domain}")
    plt.savefig(f"plots/heatmap_train-stats_{model}_ipc2023-learning-{domain}.png")
    plt.clf()

/tmp/ipykernel_5863/3297856540.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1, 3)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

2D scatter

In [82]:
import matplotlib as mpl
import matplotlib.cm as cm

cmap=cm.rainbow(np.array(ITERATIONS)/np.max(ITERATIONS)) 

for domain in IPC2023_LEARNING_DOMAINS:
  for model in ["linear-svr", "lasso", "ridge"]:
    d = get_data(domain)

    map_scores = {}
    for k, config in enumerate(d['config']):
      if model not in config:
        continue
      toks = config.split("_")
      key = (int(toks[-2]), int(toks[-1]))
      map_scores[key] = {
        "mse": d['mse'][k],
        "f1": d['f1'][k],
        "nonzero_weights": d['nonzero_weights'][k],
      }

    xs = {i: [] for i in ITERATIONS}
    ys = {i: [] for i in ITERATIONS}
    for metric in ["mse", "nonzero_weights"]:
      for i in ITERATIONS:
        for j in PRUNE_COEFS:
          try:
            if metric=="mse":
              ys[i].append(map_scores[(i, j*i)][metric])
            else:
              xs[i].append(map_scores[(i, j*i)][metric])
          except:
            pass
    
    lines = []
    for i in ITERATIONS:
      plt.scatter(xs[i], ys[i], label=i, c=cmap[i-1])
    plt.xscale("log")
    plt.xlabel("nonzero_weights (log)")
    plt.ylabel("mse")
    plt.legend()
    plt.ylim(bottom=0)
    plt.title(f"{model} {domain}")
    plt.savefig(f"plots/scatter_train-stats_{model}_ipc2023-learning-{domain}.png")
    plt.clf()

/tmp/ipykernel_5863/2250067221.py:37: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  plt.scatter(xs[i], ys[i], label=i, c=cmap[i-1])


<Figure size 640x480 with 0 Axes>

In [72]:
for domain in IPC2023_LEARNING_DOMAINS:
  print(domain, N_TRAINING_DATA[domain])
  df = get_df(domain)
  display(df)

blocksworld 4954


,config,mse,f1,nonzero_weights
0,linear-svr_wl_1_0,0.32,0.45,6150
1,linear-svr_wl_1_1,0.62,0.42,5268
2,linear-svr_wl_1_2,0.65,0.45,1887
3,linear-svr_wl_1_3,0.58,0.42,1644
4,linear-svr_wl_1_4,0.68,0.36,584
...,...,...,...,...
255,lasso_wl_9_45,3.74,0.23,7
256,lasso_wl_9_54,3.74,0.23,7
257,lasso_wl_9_63,3.74,0.23,7
258,lasso_wl_9_72,3.74,0.23,7


childsnack 2148


,config,mse,f1,nonzero_weights
0,linear-svr_wl_1_0,0.11,0.91,1113
1,linear-svr_wl_1_1,0.34,0.51,1057
2,linear-svr_wl_1_2,0.11,0.91,952
3,linear-svr_wl_1_3,0.13,0.91,909
4,linear-svr_wl_1_4,0.18,0.89,824
...,...,...,...,...
265,lasso_wl_9_45,1.46,0.38,8
266,lasso_wl_9_54,1.46,0.38,8
267,lasso_wl_9_63,1.46,0.38,8
268,lasso_wl_9_72,1.46,0.38,8


ferry 3662


...

,config,mse,f1,nonzero_weights
0,linear-svr_wl_1_0,42.05,0.03,201
1,linear-svr_wl_1_1,34.73,0.05,199
2,linear-svr_wl_1_2,34.73,0.05,199
3,linear-svr_wl_1_3,34.73,0.05,199
4,linear-svr_wl_1_4,35.39,0.05,198
...,...,...,...,...
265,lasso_wl_9_45,27.32,0.05,400
266,lasso_wl_9_54,27.32,0.05,400
267,lasso_wl_9_63,27.32,0.05,400
268,lasso_wl_9_72,27.32,0.05,400


spanner 1416


,config,mse,f1,nonzero_weights
0,linear-svr_wl_1_0,0.71,0.49,545
1,linear-svr_wl_1_1,0.71,0.48,519
2,linear-svr_wl_1_2,0.71,0.49,498
3,linear-svr_wl_1_3,0.72,0.49,464
4,linear-svr_wl_1_4,0.71,0.45,445
...,...,...,...,...
265,lasso_wl_9_45,1.83,0.24,4
266,lasso_wl_9_54,1.83,0.24,4
267,lasso_wl_9_63,1.83,0.24,4
268,lasso_wl_9_72,1.83,0.24,4


transport 4316


,config,mse,f1,nonzero_weights
0,linear-svr_wl_1_0,33.52,0.19,878
1,linear-svr_wl_1_1,33.46,0.18,855
2,linear-svr_wl_1_2,34.04,0.18,748
3,linear-svr_wl_1_3,33.75,0.17,689
4,linear-svr_wl_1_4,33.90,0.16,672
...,...,...,...,...
257,lasso_wl_9_45,43.53,0.04,13
258,lasso_wl_9_54,43.53,0.04,13
259,lasso_wl_9_63,43.53,0.04,13
260,lasso_wl_9_72,43.53,0.04,13


### WL metrics

In [73]:
def get_models(domain):
  d = {}

  for iterations, prune in itertools.product(ITERATIONS, PRUNE_COEFS):
    model = "linear-svr"
    log_file = "_".join([model, "llg", "ipc2023-learning-"+domain, "wl", str(iterations), str(prune*iterations)])+".log"
    log_file = _LOG_DIR + "/" + log_file
    
    if not os.path.exists(log_file):
      continue
      
    print(domain, iterations, prune)
    
    model_file = None
    for line in list(open(log_file, 'r').readlines()):
      toks = line.split()
      if "Model parameter file:" in line and len(toks)>3:
        model_file = line.split()[-1]
        break
    
    if model_file is None:
      continue
    model : KernelModelWrapper = load_kernel_model(model_file)[0]
    d[(iterations, prune)] = model

  return d

In [74]:
# load all models
model_domain = {}
for domain in IPC2023_LEARNING_DOMAINS:
  print(domain, N_TRAINING_DATA[domain])
  models = get_models(domain)
  model_domain[domain] = models

blocksworld 4954
blocksworld 1 0


/home/dillon/.conda/envs/torch/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator LinearSVR from version 1.3.0 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


blocksworld 1 1
blocksworld 1 2
blocksworld 1 3
blocksworld 1 4
blocksworld 1 5
blocksworld 1 6


FileNotFoundError: [Errno 2] No such file or directory: 'trained_models_kernel/linear-svr_llg_ipc2023-learning-blocksworld_wl_1_6.joblib'

In [ ]:
domain = "blocksworld"
os.makedirs("plots", exist_ok=True)
for iterations, prune in itertools.product(ITERATIONS, PRUNE_COEFS):
  try:
    prune = iterations * prune
    m = model_domain[domain][(iterations, prune)]
    h = m.get_hash()
    n_colours = len(m.get_hash())
    hist = np.zeros(n_colours)
    for col, cnt in m._kernel._train_histogram.items():
      key = str(col)
      for symbol in [")", "(", " "]:
        key = key.replace(symbol, "")
      if key not in h:
        continue
      c = h[key]
      hist[c] += cnt
    hist = sorted(hist, reverse=True)
    bins = np.arange(len(hist)+1)
    plt.hist(bins[:-1], bins, weights=hist, log=True)
    plt.savefig(f"plots/wl_count_{domain}_{iterations}_{prune}_{n_colours}.png")
    plt.clf()
  except:
    pass

<Figure size 640x480 with 0 Axes>

In [ ]:
domain="blocksworld"
iterations=1
prune=0
m = model_domain[domain][(iterations, prune)]
h1 = m.get_hash()

In [ ]:
domain="blocksworld"
iterations=5
prune=5
m = model_domain[domain][(iterations, prune)]
h2 = m.get_hash()

In [ ]:
common_colours = set(h1.keys()).intersection(set(h2.keys()))
print(len(common_colours), len(h1), len(h2))

200 6219 7743


In [ ]:
model_domain[domain]

{(1, 0): <kernels.wrapper.KernelModelWrapper at 0x7fa3ab09c730>,
 (1, 1): <kernels.wrapper.KernelModelWrapper at 0x7fa3ab09c3d0>,
 (1, 2): <kernels.wrapper.KernelModelWrapper at 0x7fa3e44b9f60>,
 (1, 3): <kernels.wrapper.KernelModelWrapper at 0x7fa3ab09f0d0>,
 (1, 4): <kernels.wrapper.KernelModelWrapper at 0x7fa3a46109d0>,
 (1, 5): <kernels.wrapper.KernelModelWrapper at 0x7fa3a4611e70>,
 (3, 0): <kernels.wrapper.KernelModelWrapper at 0x7fa3a47ecd30>,
 (3, 1): <kernels.wrapper.KernelModelWrapper at 0x7fa3a46133d0>,
 (3, 2): <kernels.wrapper.KernelModelWrapper at 0x7fa3a47ee410>,
 (3, 3): <kernels.wrapper.KernelModelWrapper at 0x7fa3a47eee00>,
 (3, 4): <kernels.wrapper.KernelModelWrapper at 0x7fa3a47ec6a0>,
 (3, 5): <kernels.wrapper.KernelModelWrapper at 0x7fa3a47ec9d0>,
 (5, 1): <kernels.wrapper.KernelModelWrapper at 0x7fa3a47ec0d0>,
 (5, 2): <kernels.wrapper.KernelModelWrapper at 0x7fa3a47ec100>,
 (5, 3): <kernels.wrapper.KernelModelWrapper at 0x7fa3beeee5c0>,
 (5, 4): <kernels.wrapper

In [ ]:
from util.save_load import load_kernel_model_and_setup
m = load_kernel_model_and_setup("trained_models_kernel/linear-svr_llg_ipc2023-learning-blocksworld_wl_1_0", "../benchmarks/ipc2023-learning-benchmarks/blocksworld/domain.pddl", "../benchmarks/ipc2023-learning-benchmarks/blocksworld/training/easy/p01.pddl")
m._kernel._train_histogram

llg created!
time taken: 0.1975s
num nodes: 58
num edges: 100
graph density: 0.060496067755595885
123
